In [1]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

##### Загружаем датасет

In [2]:
df = pd.read_csv('data/insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


##### Проверяем на наличие пропусков и типы данные 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


##### Разделение признаков и целевой переменной

In [4]:
x = df.drop(['charges'], axis=1)
y = df['charges'].values

##### Преобразование категориальных признаков в числовые значения и нормализация числовых признаков

In [5]:
x['sex'] = x['sex'].map({'male': 0, 'female': 1})
x['smoker'] = x['smoker'].map({'yes': 1, 'no': 0})
x['region'] = x['region'].map({'southwest': 0, 'southeast': 1, 'northwest': 2, 'northeast': 3})

for feature in ['age', 'bmi', 'children', 'region']:
    x[feature] = (x[feature] - x[feature].mean()) / x[feature].std()
    
x.head()

,age,sex,bmi,children,smoker,region
0,-1.438227,1,-0.453151,-0.908274,1,-1.343402
1,-1.509401,0,0.509431,-0.078738,0,-0.438331
2,-0.797655,0,0.383164,1.580335,0,-0.438331
3,-0.441782,0,-1.305043,-0.908274,0,0.466741
4,-0.512957,0,-0.292447,-0.908274,0,0.466741


##### Выполняем снижение размерности на основе МГК (мин уровень дисперсии 0.95)

In [6]:
pca = PCA(n_components=0.98) # при 0.95 растет полиномиальная линейная модель 
x = pca.fit_transform(x)
x_base = x

##### Выполняем расширение базовых признаков за счет применения полиномиальной модели (степень = 3)

In [7]:
poly = PolynomialFeatures(degree=3, include_bias=False)
x_poly = poly.fit_transform(x_base)

##### Разбиваем датасеты (базовый и расширенный) на 2 множества (обучающее и тестовое).

In [8]:
# Разделение базового набора данных на тренировочные и тестовые
x_base_train, x_base_test, y_base_train, y_base_test = train_test_split(
    x_base, y, test_size=0.3, shuffle=True
)

# Разделение расширенного набора данных на тренировочные и тестовые
x_poly_train, x_poly_test, y_poly_train, y_poly_test = train_test_split(
    x_poly, y, test_size=0.3, shuffle=True
)

##### Выполняем обучение линейной и полиномиальной моделей аналитическим методом - не используя готовое библиотечное решение.


In [9]:
def analytical_solution(x, y):
    x = np.c_[np.ones(x.shape[0]), x] # Добавляем столбец единиц для учета интерсепта
    theta = np.linalg.inv(x.T @ x) @ x.T @ y
    return theta

In [10]:
# Обучение базовой модели
linear_theta_base = analytical_solution(x_base_train, y_base_train)
print("Коэффициенты базовой модели:", linear_theta_base.flatten())

# Обучение расширенной модели
linear_theta_poly = analytical_solution(x_poly_train, y_poly_train)
print("Коэффициенты расширенной модели:", linear_theta_poly.flatten())

Коэффициенты базовой модели: [13016.44268723  2817.19196864  1996.52537112 -1744.2868112
   447.81179575 -3945.45124815 23333.06197111]
Коэффициенты расширенной модели: [ 4.48043970e+18 -1.51388413e+18  1.65182095e+18  1.82816065e+18
  7.88432367e+16 -1.26394611e+18  8.00700095e+18  4.47582042e+15
  1.10030746e+16 -4.13434689e+16 -2.96261439e+16  5.80420568e+17
 -2.01944443e+18  3.26475479e+15 -3.26775132e+16 -6.20232123e+15
  9.23037867e+16 -6.96719431e+17  5.86628858e+16  5.34424082e+16
 -7.83736333e+17  6.11209602e+18 -1.30530487e+16  6.83480733e+17
 -1.66146860e+17 -9.09560914e+18  1.56357000e+18 -1.35946695e+19
  8.46884255e+14 -2.28995766e+15  1.24651906e+15  3.68275258e+15
 -1.03012427e+17 -3.84117486e+16  1.57515319e+15 -4.44789082e+15
 -1.02187655e+16  2.75384802e+17  8.57341678e+15  1.65613161e+15
  5.01493516e+15 -1.22706585e+17  1.12468948e+17  5.00559807e+15
 -2.65671441e+17  3.02371849e+16  3.56626066e+18 -3.07045013e+16
  3.66450032e+18 -2.40618097e+14  9.67646254e+14  3

##### Выполняем обучение линейной и полиномиальной моделей градиентным спуском - не используя готовое библиотечное решение.


In [11]:
def gradient_descent(x, y, learning_rate=0.01, iterations=1000):
    m, n = x.shape
    x = np.c_[np.ones(m), x]  # Добавляем столбец единиц для интерсепта
    theta = np.zeros((n + 1,))  # Инициализация коэффициентов
    for i in range(iterations):
        predictions = x @ theta
        error = predictions - y
        theta -= (learning_rate / m) * (x.T @ error)
    return theta

In [12]:
# Обучение базовой модели
gradient_theta_base = gradient_descent(x_base_train, y_base_train, learning_rate=0.0001, iterations=100000)
print("Коэффициенты базовой модели:", gradient_theta_base.flatten())

# Обучение расширенной модели
gradient_theta_poly = gradient_descent(x_poly_train, y_poly_train, learning_rate=0.0001, iterations=100000)
print("Коэффициенты расширенной модели:", gradient_theta_poly.flatten())

Коэффициенты базовой модели: [13037.4498376   2777.28598098  1968.76207493 -1694.55472632
   432.07598114 -3560.38873663 18765.57320296]
Коэффициенты расширенной модели: [ 8.85138825e+03  2.23010119e+03  1.70264962e+03 -1.02347839e+03
  7.12714036e+02 -1.16398209e+03  7.00010209e+03  4.05149783e+02
  3.38250104e+02 -2.39144029e+02 -7.48051913e+02 -1.35921444e+03
  3.93764248e+03  8.77643600e+02  1.76139845e+02  2.35665635e+01
  1.52981701e+02 -1.50416963e+03  1.00943470e+03  6.06558252e+02
 -1.25624635e+02 -1.36423623e+03  6.86300046e+02 -8.36676224e+02
  3.43716469e+03  2.55431041e+03 -1.39139970e+03  5.20188900e+03
  4.16721248e-01  3.47232608e+02 -4.37490784e+01 -3.22541321e+02
 -4.24983495e+02  2.55148509e+03  5.59716701e+02  1.59161293e+00
 -1.08327216e+02  2.78743662e+02  1.05611715e+02 -3.80341329e+02
  3.64830432e+02 -5.26318306e+02  4.56693251e+02 -4.65313641e+02
  1.51407514e+01 -1.27698097e+03  8.33214033e+02 -1.08151090e+03
  2.50269700e+03  5.04436966e+01 -5.12328150e+02 -

In [13]:
def predict(x, theta):
    x = np.c_[np.ones(x.shape[0]), x]  # Добавляем столбец единиц для интерсепта, если его нет
    return x @ theta

# Предсказания для базовой модели
y_base_pred_linear = predict(x_base_test, linear_theta_base)
y_base_pred_gradient = predict(x_base_test, gradient_theta_base)

# Предсказания для полиномиальной модели
y_poly_pred_linear = predict(x_poly_test, linear_theta_poly)
y_poly_pred_gradient = predict(x_poly_test, gradient_theta_poly)

print("Метрики для базовой модели (Linear):")
print(f"MAE : {mean_absolute_error(y_base_test, y_base_pred_linear):.3f}")
print(f"MSE : {mean_squared_error(y_base_test, y_base_pred_linear):.3f}")
print(f"R²  : {r2_score(y_base_test, y_base_pred_linear):.3f}")

print("\nМетрики для полиномиальной модели (Linear):")
print(f"MAE : {mean_absolute_error(y_poly_test, y_poly_pred_linear):.3f}")
print(f"MSE : {mean_squared_error(y_poly_test, y_poly_pred_linear):.3f}")
print(f"R²  : {r2_score(y_poly_test, y_poly_pred_linear):.3f}")

print("\nМетрики для базовой модели (Gradient):")
print(f"MAE : {mean_absolute_error(y_base_test, y_base_pred_gradient):.3f}")
print(f"MSE : {mean_squared_error(y_base_test, y_base_pred_gradient):.3f}")
print(f"R²  : {r2_score(y_base_test, y_base_pred_gradient):.3f}")

print("\nМетрики для полиномиальной модели (Gradient):")
print(f"MAE : {mean_absolute_error(y_poly_test, y_poly_pred_gradient):.3f}")
print(f"MSE : {mean_squared_error(y_poly_test, y_poly_pred_gradient):.3f}")
print(f"R²  : {r2_score(y_poly_test, y_poly_pred_gradient):.3f}")

Метрики для базовой модели (Linear):
MAE : 4626.667
MSE : 46572520.118
R²  : 0.716

Метрики для полиномиальной модели (Linear):
MAE : 24200.550
MSE : 1112706882.143
R²  : -6.813

Метрики для базовой модели (Gradient):
MAE : 4994.318
MSE : 50798276.181
R²  : 0.691

Метрики для полиномиальной модели (Gradient):
MAE : 3542.239
MSE : 32325540.171
R²  : 0.773
